## 注意力机制Attention:

### 简介:

这一部分是Transformer模型的核心部分,以下部分逐步给出实现过程中可能用到的一些矩阵运算的原理， 以下代码均不需要大家实现,希望大家阅读代码以及下列文档中的信息:

https://arxiv.org/abs/1706.03762

https://jalammar.github.io/illustrated-transformer/

理解Attention的运行机制以及实现过程的数学技巧，完成最后的主文件中的HeadAttention(),MultiHeadAttention()部分。

我们虚构一组输入数据的Embedding用于这部分讲解：

In [36]:
import torch
from torch import nn
import torch.nn.functional as F
B, T, C = 1, 8, 16   ## B: batch size 一次训练的数据量, T: context length 前文token数, C: embedding length 隐变量长度
inputData = torch.rand(size=(B,T,C))

for i in range(T):
    print(f"Embedding of {i}th position:\n {inputData[0,i]}")


Embedding of 0th position:
 tensor([0.9763, 0.0072, 0.9507, 0.6052, 0.0376, 0.4555, 0.4017, 0.8734, 0.0435,
        0.9768, 0.7459, 0.1682, 0.3900, 0.1892, 0.7500, 0.0047])
Embedding of 1th position:
 tensor([0.1393, 0.1893, 0.6977, 0.7756, 0.7860, 0.6277, 0.1492, 0.7115, 0.8659,
        0.1821, 0.0199, 0.0396, 0.1383, 0.9170, 0.9527, 0.3319])
Embedding of 2th position:
 tensor([0.0631, 0.0081, 0.3270, 0.2026, 0.1454, 0.3984, 0.3232, 0.1445, 0.9950,
        0.5391, 0.1589, 0.6116, 0.0015, 0.6585, 0.5287, 0.7023])
Embedding of 3th position:
 tensor([0.2515, 0.3297, 0.9116, 0.3622, 0.5213, 0.6461, 0.9528, 0.2679, 0.8759,
        0.5687, 0.9841, 0.8221, 0.5582, 0.0774, 0.9798, 0.4803])
Embedding of 4th position:
 tensor([0.5399, 0.3748, 0.5374, 0.0127, 0.5118, 0.7786, 0.0178, 0.1556, 0.2836,
        0.4847, 0.7265, 0.6653, 0.1853, 0.4098, 0.7970, 0.1967])
Embedding of 5th position:
 tensor([0.1192, 0.1424, 0.8270, 0.1818, 0.7434, 0.7955, 0.8563, 0.4411, 0.1274,
        0.5730, 0.2393, 0.3

Attention从直观上可以理解为对前文各个位置信息的融合以获得当前语境所需的信息。 一个最简单的融合方式为对前文Embedding加权求和作为当前位置的信息。

我们计算第i个位置的融合后的embedding:

假设前i个位置的embedding的权重相同，均为1/i，即更新后第i个位置embedding为前文所有位置embedding的平均值：

In [21]:
def Attention_version1(contextEmbeddings):
    for i in range(T):
        context_embeddings = contextEmbeddings[0,:i+1,:] ## shape [i+1, C]
        # print(context_embeddings)
        new_embedding_for_i = torch.mean(context_embeddings,dim=0)
        contextEmbeddings[0,i] = new_embedding_for_i
    return contextEmbeddings

print("Embedding of Data after aggregate context embedding:\n", Attention_version1(inputData))

Embedding of Data after aggregate context embedding:
 tensor([[[0.3532, 0.2544, 0.5385, 0.9351, 0.8330, 0.5460, 0.4041, 0.6652,
          0.6192, 0.2962, 0.9570, 0.8507, 0.1769, 0.1493, 0.0446, 0.9899],
         [0.3572, 0.2629, 0.5306, 0.9299, 0.8329, 0.5467, 0.4132, 0.6572,
          0.6112, 0.3035, 0.9539, 0.8409, 0.1864, 0.1611, 0.0538, 0.9850],
         [0.3577, 0.2621, 0.5318, 0.9302, 0.8319, 0.5467, 0.4127, 0.6573,
          0.6121, 0.3032, 0.9535, 0.8409, 0.1862, 0.1600, 0.0538, 0.9844],
         [0.3575, 0.2623, 0.5318, 0.9301, 0.8321, 0.5466, 0.4126, 0.6572,
          0.6120, 0.3031, 0.9535, 0.8409, 0.1861, 0.1602, 0.0538, 0.9844],
         [0.3576, 0.2623, 0.5318, 0.9301, 0.8321, 0.5466, 0.4126, 0.6573,
          0.6120, 0.3032, 0.9535, 0.8409, 0.1861, 0.1602, 0.0538, 0.9845],
         [0.3575, 0.2623, 0.5318, 0.9301, 0.8321, 0.5466, 0.4126, 0.6573,
          0.6120, 0.3032, 0.9535, 0.8409, 0.1861, 0.1602, 0.0538, 0.9844],
         [0.3575, 0.2623, 0.5318, 0.9301, 0.8321, 0.

我们将上述的mean操作换为等价的矩阵运算，以i=3 为例：

new_embedding_for_3 = torch.mean(contextEmbeddings[0,:3+1],dim=0)

等价于(@ 是矩阵乘法):

new_embedding_for_3 = contextEmbeddings[0] @ torch.tensor([1/4,1/4,1/4,1/4,0,0,0,0])

In [18]:
def Attention_version2(contextEmbeddings):
    for i in range(T):
        weight = torch.cat( (torch.ones(i+1) / (i+1), torch.zeros(T-i-1,dtype=torch.float)),dim=0)
        contextEmbeddings[0,i] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version2: ",torch.all(Attention_version1(inputData) == Attention_version2(inputData)).item())

Attention_version1 equivalent to Attention_version2:  True


接下来我们用矩阵运算进一步简化上述运算，移除其中的for循环:

其中 weight = torch.tril(torch.ones(T,T)) 得到:

[[1., 0., 0., 0., 0., 0., 0., 0.],

 [1., 1., 0., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 1.]]
 
表示前文的求和权重相同都为一。

weight = weight.masked_fill(weight==0,float("-inf"))

weight = F.softmax(weight)

这两行用于归一化weight,即每一次加权求和的权重和为1，具体详见Softmax公式,我们可得到：

[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],

[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],

[0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],

[0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]


In [31]:
def Attention_version3(contextEmbeddings):
    B, T, C = contextEmbeddings.shape
    weight = torch.tril(torch.ones(T,T))
    print("weight of context embeddings:\n",weight)
    weight = weight.masked_fill(weight==0,float("-inf"))
    weight = F.softmax(weight,dim=1)
    print("weight of context embeddings after regularization:\n",weight)
    contextEmbeddings[0] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version3: ",torch.all(Attention_version1(inputData) == Attention_version3(inputData)).item())

weight of context embeddings:
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
weight of context embeddings after regularization:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.125

最后，我们确定计算weight的方法，上述三个版本的weight都是假定所有前文信息的重要程度相同,在大语言模型中，我们希望有一个灵活的方式计算前文信息对应当前语境的重要程度，为此Transformer引入了Query，Key，Value:

其中Query可以理解为当前语境对于前文信息的需求，Key可以理解为前文包含信息的索引，Value为前文所包含的信息。

Query 和 Key 用来计算信息融合的weight.

如何计算Query和Key，并用他们计算weight对Value加权求和是这次实验的重点内容，这里不能给出大家具体代码，希望大家参见Attention is All you need原论文以及助教提供的文档最后的参考链接学习这部分。

利于Query和Key得出的是信息相关性，我们需要遮盖住下文的信息(生成第i个token时，只可以使用0到i-1处的信息)，并且要对相关性归一化使之可以作为weight。这里利于Attension_version3()中的结论给出如何对计算出来的相关性加掩码和归一化:


In [40]:
def weight_mask_and_normalization(weight):
    tril = torch.tril(torch.ones_like(weight))
    weight = weight.masked_fill(tril == 0, float("-inf"))
    weight = F.softmax(weight,dim=-1)
    return weight

weight = torch.rand(T,T)
print("weight before mask and normalize:\n",weight)
print("weight after mask and normalize:\n",weight_mask_and_normalization(weight))

weight before mask and normalize:
 tensor([[0.7870, 0.0254, 0.1334, 0.4008, 0.0778, 0.4283, 0.3402, 0.2388],
        [0.0968, 0.7132, 0.4312, 0.1972, 0.8224, 0.7568, 0.3164, 0.0165],
        [0.2631, 0.2610, 0.9398, 0.5731, 0.9546, 0.7546, 0.2157, 0.7245],
        [0.7475, 0.0019, 0.8504, 0.6116, 0.2733, 0.0128, 0.1459, 0.9757],
        [0.0822, 0.7767, 0.4533, 0.0313, 0.9258, 0.3594, 0.3122, 0.8337],
        [0.0401, 0.0792, 0.7852, 0.4147, 0.5901, 0.5907, 0.9240, 0.0364],
        [0.6337, 0.7870, 0.2391, 0.5619, 0.4029, 0.9759, 0.8234, 0.3456],
        [0.3499, 0.1808, 0.2455, 0.5295, 0.0214, 0.1958, 0.2158, 0.5388]])
weight after mask and normalize:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3506, 0.6494, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2522, 0.2517, 0.4961, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2894, 0.1373, 0.3207, 0.2526, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1294, 0.2592, 0.1876, 0.1230, 0